# Using Time Series to more accurately predict wave heights

In [2]:
import utilities.misc as misc
import utilities.metrics as mtr
import utilities.ml_functions as ml
import pandas as pd

In [6]:
# convert datetime to unix
df = pd.read_csv('clean_weather/valid_dates/m1_valid_dates.csv', parse_dates=['date', 'time'])
start_date = min(df['time'])
start_date

In [ ]:
# plot data in time 

In [ ]:
# predict using daily seasonality